# The LogCA model

This notebook constructs the [LogCA model](https://dl.acm.org/doi/pdf/10.1145/3140659.3080216) to analytically model simple host-accelerator interaction

## LogCA parameters

| Parameter                  | Description                                       | Units |
|:---------------------------|:--------------------------------------------------|-------|
|$L$ - Latency             | Cycles to move data from the host to the accelerator across the interface, including the cycles data spends in the caches or memory | Cycles |
|$o$ - Overhead            | Cycles the host spends in setting up the algorithm | Cycles |
|$g$ - Granularity         | Size of the offloaded data | Bytes |
|$C$ - Computational Index | Cycles the host spends per byte of data | Cycles/Byte |
|$A$ - Acceleration        | The peak speedup of an accelerator | N/A |

### Other parameters:
- ```complexity_power_factor``` ($ \beta $) is the power factor of the complexity of the alogrithm as a function of granularity as per the model.
- ```is_latency_granularity_depend``` indicates whether the latency is dependant on granularity.

Compute the speedup for given LogCA parameters. 

In [53]:
def speedup(latency, 
            overhead, 
            granularity, 
            computational_index, 
            acceleration, 
            complexity_power_factor, 
            is_latency_granularity_depend):
    
    host_comp_index = computational_index * (granularity ** complexity_power_factor)
    
    if(is_latency_granularity_depend):
        latency = latency * granularity
            
    return host_comp_index / (latency + overhead + (host_comp_index / acceleration))

Compute break-even granularity $ g_1 $. It is the granularity required to achieve a speedup of 1.

In [54]:
def break_even_granularity(latency, 
                           overhead, 
                           computational_index,
                           acceleration,
                           complexity_power_factor,
                           is_latency_granularity_depend):
    
    if(is_latency_granularity_depend):
        return ((computational_index * (complexity_power_factor - 1) * (acceleration - 1) + acceleration * overhead) / \
               (computational_index * complexity_power_factor * (acceleration - 1) - acceleration * latency))
    
    return ((acceleration / (acceleration - 1)) * ((overhead + latency) / computational_index)) ** (1 / complexity_power_factor)

Compute half-speedup granularity $ g_{A/2} $.

In [55]:
def half_speedup_granularity(latency, 
                             overhead, 
                             computational_index,
                             acceleration, 
                             complexity_power_factor,
                             is_latency_granularity_depend):
    
    if(is_latency_granularity_depend):
        return ((computational_index * (complexity_power_factor - 1) + acceleration * overhead) / \
               (computational_index * complexity_power_factor - acceleration * latency))
    
    return (acceleration * ((overhead + latency) / computational_index)) ** (1 / complexity_power_factor)

def half_speedup_granularity_from_beg(be_granularity,
                             acceleration,
                             complexity_power_factor):
    
    return ((acceleration - 1) ** (1 / complexity_power_factor)) * be_granularity

### Example system

The below parameters model an APU with FFT and APU kernels as described in the paper.

In [66]:
latency = 15
overhead = 4 * 10**8
acceleration = 7

# Speedup achieved with accelerator at break-even granularity must be 1
# Speedup achieved with accelerator at half-speedup granularity must be A/2 = 3.5

# FFT
computational_index_fft = 290
complexity_power_factor_fft = 1.2

beg_fft = break_even_granularity(latency, overhead, computational_index_fft, acceleration, complexity_power_factor_fft, False)
hsg_fft = half_speedup_granularity(latency, overhead, computational_index_fft, acceleration, complexity_power_factor_fft, False)

print("FFT kernel on APU: ")
print("Break-even granularity: ", beg_fft)
print("Speedup achieved with accelerator at break-even granularity: ", speedup(latency, overhead, beg_fft, computational_index_fft, acceleration, complexity_power_factor_fft, False))
print("Half-speedup granularity: ", hsg_fft)
print("Speedup achieved with accelerator at half-speedup granularity: ", speedup(latency, overhead, hsg_fft, computational_index_fft, acceleration, complexity_power_factor_fft, False)) 

# AES
computational_index_aes = 174
complexity_power_factor_aes = 1

beg_aes = break_even_granularity(latency, overhead, computational_index_aes, acceleration, complexity_power_factor_aes, True)
hsg_aes = half_speedup_granularity(latency, overhead, computational_index_aes, acceleration, complexity_power_factor_aes, True)

print("\nAES kernel on APU: ")
print("Break-even granularity: ", beg_aes)
print("Speedup achieved with accelerator at break-even granularity: ", speedup(latency, overhead, beg_aes, computational_index_aes, acceleration, complexity_power_factor_aes, True))
print("Half-speedup granularity: ", hsg_aes)
print("Speedup achieved with accelerator at half-speedup granularity: ", speedup(latency, overhead, hsg_aes, computational_index_aes, acceleration, complexity_power_factor_aes, True)) 

FFT kernel on APU: 
Break-even granularity:  148653.1099524985
Speedup achieved with accelerator at break-even granularity:  0.9999999999999999
Half-speedup granularity:  661657.7058444184
Speedup achieved with accelerator at half-speedup granularity:  3.5000000000000004

AES kernel on APU: 
Break-even granularity:  2981895.633652822
Speedup achieved with accelerator at break-even granularity:  1.0
Half-speedup granularity:  40579710.14492754
Speedup achieved with accelerator at half-speedup granularity:  3.5


A simple experimental model to model the performance of W-projection gridding algorithm.

In [57]:
import time

I = 10
J = 10
K = 10
L = 2

(DISCRETE, INTEGRATED) = (True, False)

def __gridding_model__(in_latency, processing_delay, out_latency):
    # Adjust the latency and delay numbers according to SDP Memo
    # Please add more loops as necessary
    time.sleep(in_latency)
    m = 0
    for i in range(1,I):
        for j in range(1,J):
            for k in range(1,K):
                for l in range (1,L):
                    time.sleep(processing_delay)
                    m+=1
    time.sleep(out_latency)
    return m

def discrete_accelerated_gridding():
    # in_latency is high, 
    # processing delay is low (as CPUs are slower than accelerators), 
    # out_latency is high 

    return __gridding_model__(1.0, 0.001, 1.0)
    
def integrated_accelerated_gridding():
    # in_latency is low, 
    # processing delay is low (as CPUs are slower than accelerators), 
    # out_latency is low
    return __gridding_model__(0.001, 0.001, 0.001)

def accelerated_gridding(mode):
    if mode == DISCRETE:
        return discrete_accelerated_gridding()
    else:
        return integrated_accelerated_gridding()

def cpu_gridding():
    # in_latency is low, 
    # processing delay is high (as CPUs are slower than accelerators), 
    # out_latency is low
   return __gridding_model__(0.001, 0.01, 0.001) 


def main():
    start = time.perf_counter()
    cpu_gridding()
    print("cpu_gridding: ", time.perf_counter() - start)

    start = time.perf_counter()
    discrete_accelerated_gridding()
    print("discrete_accelerated_gridding: ", time.perf_counter() - start)

    start = time.perf_counter()
    integrated_accelerated_gridding()
    print("integrated_accelerated_gridding: ", time.perf_counter() - start)

main()



cpu_gridding:  7.430678297008853
discrete_accelerated_gridding:  2.8401875749987084
integrated_accelerated_gridding:  0.8432921769854147
